## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto
### Phase1: Load the Postal Codes in a DataFrame

In [1]:
#Qgrid Code dont touch
import pandas as pd
import qgrid

# function to print out dataframe
def showQgrid(df):
    from IPython.display import display
    grid = qgrid.QGridWidget(df=df)
    QGridWidget = qgrid.show_grid(df.head(), show_toolbar=True)
    QGridWidget.get_changed_df()
    display(grid)

In [2]:
import urllib.request
import pandas as pd

WikiData = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").read()

In [3]:
#Load Desired Table in Data Frame

WikiTables = pd.read_html(WikiData)
df_post = WikiTables[0]

In [4]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

indexNames = df_post[ df_post['Borough'] == "Not assigned"].index
# Delete these row indexes from dataFrame
df_post.drop(indexNames , inplace=True)

## Solution 1 In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [5]:
df_post.shape

(103, 3)

### Phase2: Load the Locations in a DataFrame    

In [6]:
GeoData = pd.read_csv("https://cocl.us/Geospatial_data")

In [7]:
df_loc = pd.merge(df_post,GeoData,left_on='Postal Code',right_on='Postal Code')

## Solution 2 : Print out the Postal Code Dataframe

In [8]:
df_loc

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


### Phase3: Cluster the locations in Toronto

In [9]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [10]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_loc['Latitude'], df_loc['Longitude'], df_loc['Borough'], df_loc['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 3.1 Retrieve Venue Details from Foursquare

In [12]:
CLIENT_ID = 'UAKLSDGE4CSMD3AA53HLK524Y4ELAS00ATYDLXY41QJWMDCK' # your Foursquare ID
CLIENT_SECRET = 'IQYOKO3UWLKNKIHF3JAFDNE5SPPFWL2IIEN1BJ53GK34GYVC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
Toronto_venues = getNearbyVenues(names=df_loc['Neighbourhood'],
                                   latitudes=df_loc['Latitude'],
                                   longitudes=df_loc['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [20]:
#showQgrid(Toronto_venues)

In [16]:
# I want to get a count of venues by type in each Neighbourhood
Toronto_venues.shape

(2162, 7)

### Prepare a Dataframe with the Top 5 Neighbourhoods so that we can use for checking later

In [17]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.shape

(2162, 273)

In [21]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
num_top_venues = 10

for hood in Toronto_grouped['Neighbourhood']:
    #print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

In [22]:
import numpy as np

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

#neighborhoods_venues_sorted.head()
neighborhoods_venues_sorted.shape

(96, 11)

### Pivot the table so we get number of venues by type for each neighbourhood. Also Drop the Not Needed columns

In [23]:
# Step 1: Pivot it so that you get aggregates of each venue type in a row
Toronto_venues_Temp2 = Toronto_venues.drop(['Venue'],axis=1)
Toronto_venues_Group2 = pd.pivot_table(Toronto_venues_Temp2, 
                                       index=['Neighbourhood',
                                              'Neighbourhood Latitude', 
                                              'Neighbourhood Longitude',
                                              'Venue Category'],
                                       values=['Venue Latitude'], aggfunc=len,fill_value=0)
Toronto_venues_Group2.rename(columns={"Venue Latitude": "Venue_Counts"},inplace=True)
Toronto_venues_Group2.reset_index(inplace=True)

In [24]:
#showQgrid(Toronto_venues_Group2)

In [25]:
# Step 2 Put Venue types in columns, and one Neighbourhood should be each Row, values should be number of venues in that neighbourhood
Toronto_venues_Group3 = pd.pivot_table(Toronto_venues_Group2, 
                                       index=['Neighbourhood',
                                              'Neighbourhood Latitude', 
                                              'Neighbourhood Longitude'],
                                       columns=['Venue Category'],
                                        values=['Venue Category'], 
                                              aggfunc=np.sum,fill_value=0)
Toronto_venues_Group3.rename(columns={"Venue Category": "Venue_Counts"},inplace=True)

In [26]:
Toronto_venues_Group3.columns = Toronto_venues_Group3.columns.get_level_values(1)
Toronto_venues_Group3.reset_index(inplace=True)
#Toronto_venues_Group3.columns = [' '.join(col).strip() for col in Toronto_venues_Group3.columns.values]
Toronto_venues_Group3.rename(columns={"":"Neighbourhood","":"Neighbourhood Latitude","":"Neighbourhood Longitude"},inplace=True)

In [28]:
#The Final Dataframe ready to be clustered
Toronto_venues_Group3.head()

Venue Category,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,43.794200,-79.262029,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Alderwood, Long Branch",43.602414,-79.543484,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bayview Village,43.786947,-79.385975,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


### 3.3 Cluster the Neighborhoods

In [30]:
# set number of clusters
kclusters = 6

Toronto_grouped_clustering = Toronto_venues_Group3.drop(['Neighbourhood','Neighbourhood Latitude','Neighbourhood Longitude'], 1)
#Toronto_grouped_clustering.describe(include = 'all')
pd.options.display.max_rows = 999
#Toronto_grouped_clustering.dtypes

In [31]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([4, 4, 0, 4, 0, 3, 4, 0, 4, 4, 4, 4, 4, 2, 0, 2, 4, 4, 1, 0])

In [32]:
# add clustering labels
Toronto_venues_Group3.insert(0, 'Cluster Labels', kmeans.labels_)

In [33]:
#Toronto_venues_Group3.head()

In [34]:
Toronto_merged2 = neighborhoods_venues_sorted.join(Toronto_venues_Group3.set_index('Neighbourhood'), on='Neighbourhood')

In [35]:
#Toronto_merged2

## Solution 3 : Clustering Analysis

In [42]:
#showQgrid(Toronto_merged2)
#Toronto_merged2.head(5)

In [37]:
Toronto_merged = df_loc

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(Toronto_venues_Group3.set_index('Neighbourhood'), on='Neighbourhood')

#Toronto_merged.head() # check the last columns!
#Toronto_merged.describe(include='all')


In [38]:
Toronto_merged['Cluster Labels'] = Toronto_merged['Cluster Labels'].fillna(0)

### Distribution of Clusters

In [39]:
Toronto_merged.groupby('Cluster Labels').count()[['Neighbourhood']]

,Neighbourhood
Cluster Labels,
0.0,25
1.0,4
2.0,5
3.0,3
4.0,78
5.0,2


In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    colorlabel = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[colorlabel],
        fill=True,
        fill_color=rainbow[colorlabel],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters